# HW-10 讓車子看得見-自駕車影像資料取得與分析 (線性代數/多媒體工程)

## Tutorials

- [Lecture 2 - 3: Homogeneous Transformations ](https://www.youtube.com/watch?v=kmceHOsxFDI)

### Reference:

- [Python-Numpy Tutorial](https://cs231n.github.io/python-numpy-tutorial/)
- [Scipy-lectures: Numpy Part](https://scipy-lectures.org/intro/numpy/index.html)

- [機器人學-第一週和第二週](https://www.coursera.org/learn/robotics1#syllabus)

## Objectives:

- 理解相機座標系、Duckiebot座標系彼此相對轉換關係
- 使用Apliltags來進行Duckiebot 姿態估算
- 實現(可靠的?)**全自動停車系統**

## Project Code

### 使用Pip在自己電腦端安裝下列packages

```
pip install pupil_apriltags math3d
```

### 下載Lab10的code

- **小鴨車還有自己電腦端都需要下載code**，但之後會各自跑不同的檔案

```
git clone -b Lab10 https://github.com/GrassLab/NCTU_CS_ZOO.git Lab10
或是手動選擇Lab10的分支下載zip檔
```

### Step1: 執行新版外部參數校正:

- 開啟Terminal，使用SSH **連進小鴨車**

- 手動拷貝HW5 校正存好的**intrinsics.pkl**到 *duckiebot_cs_zoo/app/data/camera_calibration*下面

- 跑新版的外部參數校正(如果畫面無法出現請找助教XD)

  - 將小鴨車放在校正位置 (參考 [Challenge 5 Step4](https://github.com/GrassLab/NCTU_CS_ZOO/blob/Challenge5/others/Challenge5.ipynb))

  ```
  cd duckiebot_cs_zoo
  python -m app.extrinsic_calibration_pose
  ```


- 跑完以後，會出現類似下面的輸出結果

  ```
  carTcam Position:[ 0.06612483 -0.00073346  0.10408077]
  Camera calibration saved, path:/home/YOURNAME/NCTU_CS_ZOO/duckiebot_cs_zoo/app/data/camera_calibration/camera_data.pkl
  ```

  其中，carTcam Position代表相機在小鴨車座標系下的X,Y,Z，單位為公尺。可以拿尺量一下，相機鏡頭凸出的頂部到車輪平面的位置是否約略為10.4 cm，每台車可能有些許誤差

- 連線進小鴨車，手動下載**camera_data.pkl**到**自己的電腦端存放HW10 code**的位置下的**duckiebot_cs_zoo/app/data/camera_calibration**資料夾下(若無這個資料夾請自己新增)

### Step2: 認識新架構

- 之後的 Lab10-12，Duckiebot都會變成一個**單純透過socket傳送相機圖片並接收來自電腦指令的機器**，圖片處理，運算，主程式邏輯等都會**跑在自己電腦local端的python**，可以想像成Challenge7的完整版XD。

#### 小鴨車Server端

- SSH 連進去自己的小鴨車，cd 進去到**duckiebot_cs_zoo**資料夾

  ```bash
  # 方法1
  PORT=54762 python -m nodes.duckiebot

  # 方法2:預設PORT則為54761
  python -m nodes.duckiebot
  ```

- 之後這個視窗就留著不要關閉，小鴨車現在就是一個Server，等Client(電腦端)連進去操控

#### 電腦Client端

- 使用電腦端IDE/Python，進到**duckiebot_cs_zoo**資料夾
    - Mac:
        - 方法1
          ```PORT=54762 DUCKIE=duckie-學號.local python -m app.remote_client```
        - 方法2:預設PORT則為54761
          ```DUCKIE=duckie-學號.local python -m app.remote_client```
    - Windows
        - 方法1 for Powershell
            ```
            $env:DUCKIE=duckie-學號.local
  $env:PORT=54762
            python -m app.remote_client
            ```
            
        - 方法1 for Cmd
            ```
            set DUCKIE=duckie-學號.local
            set PORT=54762
            python -m app.remote_client
            ```
       
        - 方法2 for Powershell: 預設PORT則為54761
            ```
            $env:DUCKIE=duckie-學號.local
            python -m app.remote_client
            ```

        - 方法2 for Cmd: 預設PORT則為54761
            ```
            set DUCKIE=duckie-學號.local
            python -m app.remote_client
            ```
    - Pycharm
      ![Pycharm](https://i.imgur.com/DDXq2Hq.png)

- 之後可以參考 **duckiebot_cs_zoo/app/remote_client.py**裡面的使用方法操控小鴨車

## Requirements

### 任務描述

在這次Homework中，大家要實現一個**全自動的停車系統**，可以拆解成以下兩個問題

1. 車子在哪裡(相對某個座標系$O$)，以及車要開到哪裡(停車格在座標系$O$中的位置)
2. 怎麼移動到停車格中

### 1. 車子在哪裡?

- 我們需要先定義好一個座標系，因此助教繪製了以下的停車格，其中橘色框框為車輪最終要移動到的位置，車頭則是停好後需朝向鴨子頭頂的方向(朝上)。Demo時只要能讓兩個輪胎的**最低接觸點都在橘色框框內**即可Pass，一小部分輪胎超出框框範圍仍可接受。

![tag_figure](https://i.imgur.com/RHmLBTQ.png)

- 上圖每個黑色QR-Code的東西叫做Apriltag，攝影機可以從這個tag拿到**攝影機鏡頭相對這個Tag**的**Pose**

  - 一個Pose(姿態)指的是相對一個**3維空間的位移與旋轉**
  - 通常一個pose可以使用一個特殊的**4x4矩陣**來表示，我們在Background Knowledge會再補充說明

- 左下角有一個**座標X-Y的圖**，但請特別注意，我們**實際的原點假設在0號 tag的左下角頂點處**，**正X軸朝右，正Y軸朝上**，**正Z軸朝你** (右手定則)，我們之後稱這個座標系為Tag座標系:$O$

- 車子本身也是一個座標系(Car座標系$C$)，正X軸為朝車子前方，正Y軸朝左方

- $C$與$O$的定義如下
  ![relation1](https://i.imgur.com/aOg0BC0.png)

- 小鴨車$C$相對$O$的平面轉角(以$O$來看)
  ![relation2](https://i.imgur.com/2Rd01tz.png)

- 由於攝影機視角有限，助教只會將車子的**起始位置**放置在下列三個靠近停車格處，如下圖所示，其他位置與距離則可以暫時不管，*同樣的程式(不改動任何參數下)*只要**能Pass這三種Cases即可通過**

  ![tag_view](https://i.imgur.com/5fD7ZUz.png)

#### 小鴨車上的攝影機，只要有看到任何一個或多個Tags，即可推算出目前相對$O$的位置(X-Y座標)以及相對座標系$O$正X軸的夾角

- 反之，請避免讓小鴨車攝影機移動到看不到tag的區域(或是當看不到tag時要有對應的處理方法?

### 2. 怎麼移動到停車格中

- 假設我們解決了第一個位置問題，則在**每一個時刻**，我們可以得到車子的Pose，亦即車子座標系相對tag座標系的位移與旋轉，符號上表示成$^oT_c$，一個$^oT_c$包含了
  - 位移:$^ot_c$
  - 旋轉:$^oR_c$
- 三維空間中的$^oT_c$在**實作上有很多不同的表示法**，最常見的則是使用**4x4矩陣 (Homogeneous Transform Matrix)來表示**
- 然而我們因為都在地板平面上移動，因此可以再從$^oT_c$中提取出二維的資訊: $x,y,\theta$，分別代表平面的位移與旋轉(對$O$的+Z軸，可以理解成與$O$的+X軸夾角)，在停車時，只需要注意二維的資訊即可
- 當每個時刻的$^oT_c$都能得到時，可以自行設計一套方法(?)來讓車子開到要去的停車位置，可以想像這些Tags就像是紅外線循跡的黑線一樣，只是現在這條黑線並非固定的，而是換算成相對Tag坐標系原點的**位置與角度****=> Your HW 10**

## 實作步驟

### 初始Code: app/automatic_parking.py

#### get_current_2d_pose()

- 完成座標轉換，寫完這一部份時，建議先用手移動小鴨車，確定算出來的值合理再進行下一步

#### hw10()

- 完成自動停車的任務，由於攝影機framerate不高，建議不要用太快的速度，因為Tag可能會變模糊，導致辨識不到清楚的Tag，一分鐘內能停好都可以接受 XD

當程式完成後，首先要SSH進去小鴨車的Lab10/duckiebot_cs_zoo裡面，開啟小鴨車的Server
```bash
#先設定好環境變數PORT(不設則是預設54761)
python -m nodes.duckiebot
```
再來在自己local電腦端跑```automatic_parking.py```，跑之前和上面範例電腦Client端跑app.remote_client **一樣的環境變數要設好**，再跑
```bash
#先設定好環境變數 DUCKIE 和 PORT(不設則是預設54761)
python -m app.automatic_parking
```
如果```automatic_parking.py```有更改重跑等，小鴨車上的Server可以不用管，除非遇到奇怪的連線錯誤，再重跑Server就好

## 補充:座標轉換矩陣與作業提示

In [49]:
import math3d as m3d,numpy as np
from libs.tag_utils import get_detector, detect_apriltags, get_five_points, get_pose_mapping
pose_mapping = get_pose_mapping(nx=6, ny=9, tsize=0.025,tspace=0.25)  # Map each Tag to a 3D pose in the Tag coordinate
detector = get_detector()
print(f'Pose Mapping at Tag 0=\n{pose_mapping[0]}')
print(f'Pose Mapping at Tag 23=\n{pose_mapping[23]}')

Pose Mapping at Tag 0=
[[1.     0.     0.     0.0125]
 [0.     1.     0.     0.0125]
 [0.     0.     1.     0.    ]
 [0.     0.     0.     1.    ]]
Pose Mapping at Tag 23=
[[1.      0.      0.      0.16875]
 [0.      1.      0.      0.10625]
 [0.      0.      1.      0.     ]
 [0.      0.      0.      1.     ]]


pose_mapping[0] 代表 0號tag的姿態，先看最後一排column

In [50]:
print(pose_mapping[0][:,[3]])

[[0.0125]
 [0.0125]
 [0.    ]
 [1.    ]]


代表相對Tag坐標系原點$O$ X:1.25公分，Y:1.25公分，Z:0公分，最後一個1可以不管，而左上方的旋轉矩陣

In [51]:
print(pose_mapping[0][0:3,0:3])


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


代表相對原點無旋轉角度，亦可藉由math3d來轉換成歐拉角方便理解

In [52]:
pose0=m3d.Transform(pose_mapping[0])
print(np.rad2deg(pose0.orient.to_euler('xyz')))

[ 0. -0.  0.]


分別代表對應三軸的旋轉角度，因為都在同一張紙上，因此都為0
## 攝影機看到什麼

In [53]:
import cv2
img=cv2.imread('./img.png')
cam_mat=np.array([[313.68265,0,314.64113],
                  [0,316.58896,241.45956],
                  [0,0,1]])
tag_pts = get_five_points((0, 0), half=0.0125)

In [54]:
tag_result = detect_apriltags(img, tag_pts,cam_mat, detector, return_vis=False)
print(tag_result.keys()) # 顯示偵測到的Tags
print(f'camTtag_0=\n{tag_result[0]}')

dict_keys([0, 1, 2, 3, 6, 12, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 41])
camTtag_0=
[[ 0.84338626  0.53702883 -0.01731038 -0.10148753]
 [ 0.12344851 -0.22502567 -0.96650086  0.06347229]
 [-0.5229341   0.81299661 -0.25607896  0.17226136]
 [ 0.          0.          0.          1.        ]]


我們使用camTtag_0代表由camera坐標系(約略位於鏡頭前端中央，三軸分別為往前看為+Z，往右為+X，往下為+Y)看到tag0在畫面中的位置，可以看到tag0在鏡頭的左側，因此X為負的
可以發現左上角3x3矩陣不再是Identity Matrix，因此可以使用math3d來算出來對應的角度

In [55]:
pose_camTtag_0=m3d.Transform(tag_result[0])
print(np.rad2deg(pose_camTtag_0.orient.to_euler('xyz')))

[107.4834675   31.52927184   8.32738795]


再來我們思考，要如何將攝影機看到的這個資訊，轉化成小鴨車在Tag坐標系的Pose
由於Pose轉換矩陣有一個特性，就是透過矩陣乘法可以轉換同樣物體在不同座標系中的表示法
例如：
$^AT_B=^AT_C\times^CT_B$
或
$^CT_B=(^AT_C)^-1\times ^AT_B$


A,B,C分別為不同座標系

在這次作業中，我們總共有下列的坐標系需要處理
1. 相機座標系 cam
2. Tag坐標系  $O$
3. 小鴨車座標系 $C$
4. 每個Tag本身 $tag_i$ 自己也是一個座標系，只是和 $O$ 沒有旋轉角(三軸都平行，因為都在同一張紙上)

每兩個座標系，彼此之間的轉換關係即是用4x4的Pose矩陣描述，因此會說某個B的Pose相對A，即是代表 $^AT_B$這個代表兩者相對關係的矩陣
- ```pose_mappings```儲存的資料是 $^OT_{tag_i}$
- ```detect_apriltags```回傳的是 $^{cam}T_{tag_i}$
- 小鴨車攝影機與小鴨車座標系的相對關係為 $^CT_{cam}$，由一開始的 ```extrinsic_calibration_pose``` 得到，可由```duckiebot.camera.data['cam_carTcam']```拿到，下面為助教車子的 $^CT_{cam}$值

In [56]:
#$^CT_{cam}$
cam_carTcam=np.array([[0.00534,-0.25618,0.96661,0.06612],
                    [-0.99990,0.01098,0.00844,-0.00073],
                    [-0.01278,-0.96657,-0.25610,0.10408],
                    [ 0,0,0,1]])

(重點Here!!)=> 因此，要估算出車車相對$O$的原點，我們須在get_current_2d_pose裡實作求出 $^OT_C$，亦即**進行一些矩陣乘法運算**，每一個tag都能提供一組解
因為我們有$^OT_{tag_i}$, $^{cam}T_{tag_i}$,$^CT_{cam}$
當求出$^OT_C$後，假設存在變數```car_pose```內，則可使用
```
car_pose=m3d.Transform(car_pose)
angle_degree=np.rad2deg(car_pose.orient.to_euler('xyz')[2]))
```
得到轉角，如果計算都正確，車車正面朝向車庫的角度angle_degree應該為90度左右
因為每個tag都能算出一組互相都蠻接近的解，可以再自己想要怎麼處理或利用